In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')
os.chdir("./gdrive/My Drive/NLU/Projects/project 1/rok/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
import pickle

print("tf_version:\t" + tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

from models import ModelB
from load_embedding import load_embedding

# DIRECTORIES
DATA_DIR = "./data/"
RESULTS_DIR = "./results/"
WORD_EMBEDDINGS_FILE = "wordembeddings-dim100.word2vec"
SENTENCES_TRAIN_FILE = "sentences.train"
SENTENCES_TEST_FILE = "sentences_test.txt"
SENTENCES_EVAL_FILE = "sentences.eval"
SENTENCES_CONTINUATION_FILE = "sentences.continuation"

# LANGUAGE MODEL PARAMETERS
EMBEDDING_DIM = 100
STATE_DIM = 512
VOCABULARY_SIZE = 20000
SENT_DIM = 30

# RNN PARAMETERS
BATCH_SIZE = 64
LEARNING_RATE = 0.001
MAX_GRAD_NORM = 5.0
NUM_EPOCHS = 1
KEEP_PROBS = 0.5

# VOCABULARY
with open(RESULTS_DIR + "vocabulary.pkl", "rb") as f:
    vocabulary, word_to_idx, idx_to_word = pickle.load(f)

tf_version:	1.13.1


In [0]:
EXPERIMENT = "B"

tf.reset_default_graph()

model = ModelB(vocabulary_size=VOCABULARY_SIZE, embedding_dim=EMBEDDING_DIM, state_dim=STATE_DIM, sent_dim=SENT_DIM,
               initializer=tf.contrib.layers.xavier_initializer(), pad_idx=180, tf_graph=None)

saver = tf.train.Saver()

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    # TRAINING
    session.run(model.iterator_op, {model.sentences_file: RESULTS_DIR + "X_train.ids"})

    load_embedding(session, word_to_idx, model.embedding_weight, DATA_DIR + WORD_EMBEDDINGS_FILE, EMBEDDING_DIM,
                   VOCABULARY_SIZE)

    batch_count = 0
    total_batch = 500
    while True:
        try:
            batch_loss, batch_perplexity, _ = session.run([model.loss, model.perplexity, model.optimize_op])
            epoch = 1
            if batch_count % 100 == 0:
                print("epoch: {}/{:<6}batch: {:>5}/{:<10}loss = {:<13.2f}perp = {:<13.2f}".format(epoch, NUM_EPOCHS,
                                                                                                  batch_count + 1,
                                                                                                  total_batch,
                                                                                                  batch_loss,
                                                                                                  batch_perplexity))

            batch_count += 1
            if batch_count > total_batch:
                break
        except tf.errors.OutOfRangeError:
            break

    save_path = saver.save(session, MODEL_DIR + "/experiment" + EXPERIMENT + 
                           "/experiment" + EXPERIMENT + ".ckpt")
    print("Model saved in path: %s" % save_path)
            
    # EVALUATION
    session.run(model.iterator_op, {model.sentences_file: RESULTS_DIR + "X_eval.ids"})
    eval_perplexities = np.array([], dtype=np.float32)
    batch_count = 0
    while True:
        try:
            batch_perplexities = session.run(model.perplexities)
            eval_perplexities = np.append(eval_perplexities, batch_perplexities)
            batch_count += 1
        except tf.errors.OutOfRangeError:
            break

with open(RESULTS_DIR + "groupXX.perplexity" + EXPERIMENT, "w") as f:
    for i in range(eval_perplexities.shape[0]):
        f.write("%0.3f" % eval_perplexities[i] + "\n")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.cast instead.
Loading external embeddings from ./data/wordembeddings-dim100.word2vec
<bos> not in embedding file
<eos> not in embedding file
<pad> not in embedding file
<unk> not in embedding file
19996 words out of 20000 could be loaded
epoch: 1/1     batch:     1/500       loss = 286.99       perp = 1413.96      
epoch: 1/1     batch:   101/500       loss = 66.63        perp = 28.80        
epoch: 1/1     batch:   201/500       loss = 69.24        perp = 27.05        
epoch: 1/1     batch:   301/500       loss = 60.30    